In [1]:
pip install python-telegram-bot

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade python-telegram-bot

In [3]:
pip install python-telegram-bot

In [4]:
!pip install pyTelegramBotAPI

In [ ]:
import requests
import telebot

API_KEY = '************************************'
OWM_API_KEY = '************************************'

bot = telebot.TeleBot(API_KEY)

@bot.message_handler(commands=["hi"])
def hi(message):
    bot.reply_to(message, "Hey! How's it going?")

@bot.message_handler(commands=["HELLO"])
def hello(message):
    bot.reply_to(message, "Hello there!")
    

    
@bot.message_handler(commands=["HELP"])
def help_message(message):
    response = (
        "I am a simple bot that responds to a few commands:\n"
        "/hi - Say hello to the bot\n"
        "/HELLO - Another way to greet\n"
        "/HELP - Display this help message\n"
        "/WEATHER - Get today's weather"
    )
    bot.reply_to(message, response)

@bot.message_handler(commands=["WEATHER"])
def get_weather(message):
    # Extract the city name from the user's message
    try:
        _, city = message.text.split(" ", 1)
    except ValueError:
        bot.reply_to(message, "Please provide a city name after /WEATHER. For example: /WEATHER Paris")
        return

    # Adjust the city name if needed
    weather_api_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={OWM_API_KEY}"

    try:
        response = requests.get(weather_api_url)
        
        # Check for HTTP errors
        response.raise_for_status()
        
        data = response.json()
        print(data)  # Print the API response for debugging

        if 'weather' in data:
            weather_description = data["weather"][0]["description"]
            temperature_celsius = data["main"]["temp"]
            feels_like_temperature = data["main"]["feels_like"]
            wind_speed = data["wind"]["speed"]
            pressure = data["main"]["pressure"]
            humidity = data["main"]["humidity"]
            dew_point = data["main"]["temp"] - ((100 - humidity) / 5)
            visibility = data.get("visibility", "N/A")
            
            response_text = (
                f"Today's weather in {city}:\n"
                f"Description: {weather_description}\n"
                f"Temperature: {temperature_celsius:.2f}°C\n"
                f"Feels like: {feels_like_temperature:.2f}°C\n"
                f"Wind: {wind_speed:.2f}m/s\n"
                f"Pressure: {pressure}hPa\n"
                f"Humidity: {humidity}%\n"
                f"Dew point: {dew_point:.2f}°C\n"
                
            )
            bot.reply_to(message, response_text)
        else:
            bot.reply_to(message, "Error: Weather information not available.")

    except requests.exceptions.HTTPError as errh:
        bot.reply_to(message, f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        bot.reply_to(message, f"Error Connecting: {errc}")
    except requests.exceptions.Timeout as errt:
        bot.reply_to(message, f"Timeout Error: {errt}")
    except requests.exceptions.RequestException as err:
        bot.reply_to(message, f"Error: {err}")
@bot.message_handler(commands=["start"])
def start(message):
    bot.reply_to(message, "Welcome! This is your bot. You can use commands like /hi, /HELLO, /HELP, and /WEATHER.")

# Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your Telegram bot token
bot.polling()
